Prometheus
----------

![](https://prometheus.io/assets/architecture.png)

Quelle: [Prometheus](https://prometheus.io)

- - -

[Prometheus](https://prometheus.io) is an open source system monitoring and alerting toolkit originally developed at SoundCloud.

Prometheus joined the [Cloud Native Computing Foundation](https://www.cncf.io/) in 2016 as the second hosted project after Kubernetes.

Along with two other products, the Monitoring Open Source Stack is as follows:

* **Prometheus** is a time series database and monitoring tool that is used to query metric endpoints and fetch and process the data provided by those endpoints. Data can be queried using PromQL.
* **Grafana** is a data visualization and analysis tool that allows you to create dashboards and charts for your metrics data.
* **Alertmanager**, which is usually deployed with Prometheus, forms the alert layer of the stack. It processes alerts generated by Prometheus and deduplicates (eliminates redundant data), groups and forwards them e.g. via email.


***
### Installation

The easiest way to install Prometheus is to use [helm](https://helm.io).


In [ ]:
%%bash
kubectl create namespace monitoring
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm repo update
helm install monitoring prometheus-community/kube-prometheus-stack -n monitoring

Die Ports der User Interfaces

* Grafana (Service: prometheus-operator-grafana)
* Prometheus (Service: prometheus-operator-prometheus)
* Alertmanager (Service: prometheus-operator-alertmanager)

are only accessible within the cluster. Therefore we have to unlock them first or open them to the outside:

In [ ]:
! kubectl -n monitoring patch service monitoring-grafana                      -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-prometheus   -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-alertmanager -p '{"spec": {"type": "LoadBalancer"}}'

If you are activated, we can reach you like other services via cluster IP and port:

In [ ]:
! echo "Grafana UI     : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-grafana -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Prometheus UI  : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Alertmanager UI: " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')

**Grafana UI** Login with user/password **admin/prom-operator**. On the left you can choose between a number of preparatory dashboards, e.g. Nodes.

In the **Prometheus** interface, resources can be selected using the query language [PromQL](https://prometheus.io/docs/prometheus/latest/querying/basics/), e.g. by querying `apiserver_storage_objects`.

The **Alertmanager** is used to process alerts. See [Notifikation Beispiel](https://prometheus.io/docs/alerting/notification_examples/) for an example

- - -
### Custom resources

The Prometheus [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md) provides a set of [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md)](https://kubernetes.io/docs/tasks/access-kubernetes-api/extend-api-custom-resource-definitions/) for monitoring.

* **Prometheus**, for a Prometheus instance.
* **Alertmanager**, for an Alertmanager instance
* **ServiceMonitor**, for monitoring Kubernetes services.
* **PodMonitor**, for monitoring pods.


In [ ]:
! kubectl api-resources | grep monitoring
! echo "---------------------------------"
! kubectl explain servicemonitors                   

- - -
### Monitor own services

In order for your own (micro)services to be monitored, these metrics must provide information.

To do this, the (micro)services must provide metric information.

The synchronous microservices presented at the beginning deliver these.

For this we start the microservices fresh including ServiceMonitor for Prometheus.

In [ ]:
%%bash
kubectl create namespace ms-rest
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: microservice
  labels:
    tier: microservice
spec:
  selector:
    matchLabels:
      tier: microservice
  endpoints:
  - port: web
%EOF%

echo "http://$(cat ~/work/server-ip):$(kubectl get service apache --namespace ms-rest -o=jsonpath='{ .spec.ports[0].nodePort }')"

All containers must be started for the next commands:

In [ ]:
! kubectl get all,servicemonitors --all-namespaces -l tier=microservice

Then we can use `curl` to query the metrics, e.g. from Customer.

Among other things, the following information is output:
* `counter.status.200.metrics` - number of views of /metrics
* `counter.status.200.list.html`- Number of accesses to the HTML pages of the application

In [ ]:
! curl http://customer.ms-rest:8080/metrics | grep counter

- - -
### Prometheus

We can now monitor the created microservices with Prometheus.

For this we need the appropriate rights:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: ms-kubernetes
%EOF%


And a separate Prometheus instance.

In the user interface we can then look at the monitored targets and their status.

To do this, enter `up` as a query or display the monitored microservices via pulldown -> Status -> Targets.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: Service
metadata:
  name: prometheus
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: LoadBalancer
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
%EOF%

echo "http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"

- - -
### Prometheus example

Prometheus itself provides its own example of a monitored service.

This is described in this [Blogeintrag](https://prometheus-operator.dev/docs/user-guides/getting-started/).

We start the Prometheus example for comparison purposes:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice   
spec:
  replicas: 3
  selector:
    matchLabels:
      app: example-app
  template:
    metadata:
      labels:
        app: example-app
        tier: microservice        
    spec:
      containers:
      - name: example-app
        image: fabxc/instrumented_app
        ports:
        - name: web
          containerPort: 8080
---
kind: Service
apiVersion: v1
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice    
spec:
  selector:
    app: example-app
  ports:
  - name: web
    port: 8080
%EOF%

In [ ]:
! kubectl get all,servicemonitors -n ms-rest -l tier=microservice

In [ ]:
! curl example-app.ms-rest:8080/metrics